In [1]:
import os,sys,json
from glob import glob

os.chdir('/home/avijit/projects/Awareness_vs_Unawareness')
sys.path.insert(0,"/home/avijit/projects/Awareness_vs_Unawareness/robustfairnesscode")

In [2]:
import warnings
warnings.filterwarnings('ignore')

from robustfairnesscode import data, losses, optimization, model, utils, dro_training, softweights_training
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

In [3]:
import traceback

In [4]:
def writejson(path,arr,name):
    try:
        os.mkdir(path+'preds/')
    except:
        pass
    f = open(path+'preds/'+name,'w')
    json.dump(arr,f)
    f.close()

In [5]:
# path = 'data/datasets/publiccov_tx/'
# synthfols = glob("data/datasets/publiccov_tx/synthetic/*/")
# paths = [path]
# paths.extend(synthfols)

# path = 'data/datasets/publiccov_ny/'
# synthfols = glob("data/datasets/publiccov_ny/synthetic/*/")
# paths.append(path)
# paths.extend(synthfols)

# path = 'data/datasets/publiccov_ca/'
# # synthfols = glob("data/datasets/publiccov_ca/synthetic/*/")
# paths = [path]
# # paths.extend(synthfols)

In [6]:
paths  = ['data/datasets/publiccov_ca/', 'data/datasets/employment_ca/', 'data/datasets/law_school/', 'data/datasets/diabetes/']
cnames = [['AGEP', 'SCHL', 'MAR', 'SEX', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC',
       'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'PINCP', 'ESR', 'FER', 'RAC1P',
       'PUBCOV'],
          ['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC',
       'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'SEX', 'RAC1P', 'ESR'],
          ['zfygpa', 'zgpa', 'DOB_yr', 'weighted_lsat_ugpa', 'cluster_tier',
       'family_income', 'lsat', 'ugpa', 'isPartTime', 'sex', 'race',
       'pass_bar'],
          ['race', 'sex', 'age', 'admissiontypeid', 'dischargedispositionid',
       'admissionsourceid', 'timeinhospital', 'numlabprocedures',
       'numprocedures', 'nummedications', 'numberoutpatient',
       'numberemergency', 'numberinpatient', 'diag1', 'diag2', 'diag3',
       'numberdiagnoses', 'maxgluserum', 'A1Cresult', 'metformin',
       'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
       'rosiglitazone', 'insulin', 'change', 'diabetesMed', 'readmitted']
]

In [7]:
def task(path,cols,num):
    
    print(path)

    if True:#os.path.exists(path+'preds/Softweights_pred.json') == False:
        
        train_df = pd.read_csv(path+'train.csv',header=None)

        train_df.columns = cols


#         train_df.index = train_df['SEX']

#         train_df['SEX'] = train_df['SEX'].map({'Male': 1, 'Female': 0})


        train_df = train_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

        test_df = pd.read_csv(path+'test.csv',header=None)

        test_df.columns = cols


#         test_df.index = test_df['SEX']
        
#         train_df['SEX'] = train_df['SEX'].map({'Male': 1, 'Female': 0})

        tgt = cols[-1]

        X_train = train_df.drop([tgt],axis=1)
        X_test = test_df.drop([tgt],axis=1)

        Y_train = pd.Series(train_df[tgt])
        Y_test = pd.Series(test_df[tgt])

        Y_train = pd.Series(Y_train.factorize(sort=True)[0], index=Y_train.index)
        Y_test = pd.Series(Y_test.factorize(sort=True)[0], index=Y_test.index)

        X_merged = pd.concat([X_train,X_test])

        ohe = make_column_transformer(
            (OneHotEncoder(sparse=False), X_merged.dtypes == 'object'),
            remainder='passthrough', verbose_feature_names_out=False)

        X_merged_temp  = pd.DataFrame(ohe.fit_transform(X_merged), columns=ohe.get_feature_names_out(), index=X_merged.index)

        train_df  = pd.DataFrame(ohe.transform(X_train), columns=ohe.get_feature_names_out(), index=X_train.index)
        test_df = pd.DataFrame(ohe.transform(X_test), columns=ohe.get_feature_names_out(), index=X_test.index)
        
        train_df[tgt] = Y_train
        test_df[tgt] = Y_test

        print(train_df.columns)

        LABEL_COLUMN = tgt
        FEATURE_NAMES = list(train_df.columns)
        FEATURE_NAMES.remove(LABEL_COLUMN)
        
        PROTECTED_COLUMNS = ""
        if 'SEX' in cols: 
            PROTECTED_COLUMNS = ['SEX_Female','SEX_Male']
        else:
            PROTECTED_COLUMNS = ['sex_Female','sex_Male']
        PROXY_COLUMNS = PROTECTED_COLUMNS 


        val,_,_,_ = dro_training.get_results_for_learning_rates(train_df, test_df, FEATURE_NAMES,
                                                          PROTECTED_COLUMNS, PROXY_COLUMNS, LABEL_COLUMN, num_loops = 1,constraint='tpr_and_fpr')

        vall = [float(v) for v in val[0]]

        val_m = [0 if i < 0 else 1 for i in vall]

        writejson(path,val_m,'DRO_pred_'+str(num)+'.json')

        val2,_,_,_ = softweights_training.get_results_for_learning_rates(train_df, test_df, FEATURE_NAMES, 
                                                                   PROTECTED_COLUMNS, PROXY_COLUMNS, LABEL_COLUMN, num_loops = 1, constraint='tpr_and_fpr')

        val2l = [float(v) for v in val2[0]]

        val_m2 = [0 if i < 0 else 1 for i in val2l]

        writejson(path,val_m2,'Softweights_pred_'+str(num)+'.json')

        return path, 'DRO and SW done'

In [ ]:
# import random
# random.shuffle(paths)
for path,cols in zip(paths,cnames):
    for i in range(10):
        try:
            task(path,cols,i)
        except:
            traceback.print_exc()

data/datasets/publiccov_ca/
Index(['SCHL_1 or more years of college credit, no degree',
       'SCHL_12th grade - no diploma', 'SCHL_Associate's degree',
       'SCHL_Bachelor's degree', 'SCHL_Doctorate degree',
       'SCHL_GED or alternative credential', 'SCHL_Grade 1', 'SCHL_Grade 10',
       'SCHL_Grade 11', 'SCHL_Grade 2', 'SCHL_Grade 3', 'SCHL_Grade 4',
       'SCHL_Grade 5', 'SCHL_Grade 6', 'SCHL_Grade 7', 'SCHL_Grade 8',
       'SCHL_Grade 9', 'SCHL_Kindergarten', 'SCHL_Master's degree',
       'SCHL_No schooling completed', 'SCHL_Nursery school, preschool',
       'SCHL_Professional degree beyond a bachelor's degree',
       'SCHL_Regular high school diploma',
       'SCHL_Some college, but less than 1 year', 'MAR_Divorced',
       'MAR_Married', 'MAR_Never married or under 15 years old',
       'MAR_Separated', 'MAR_Widowed', 'SEX_Female', 'SEX_Male',
       'DIS_With a disability', 'DIS_Without a disability',
       'ESP_Both parents in labor force', 'ESP_Father in the labor

In [1]:
# from multiprocessing.pool import ThreadPool as Pool

# pool = Pool(10)
# for result in pool.imap_unordered(task, paths):
#     print(result)

In [ ]:
#   Args:
#     predictions: numpy array of floats representing predictions. Predictions are treated 
#       as positive classification if value is >0, and negative classification if value is <= 0.
#     labels: numpy array of floats representing labels. labels are also treated as positive 
#       classification if value is >0, and negative classification if value is <= 0.